In [2]:
#TRIGGER_1 = hsa-miR-143-3p, TRIGGER_2 = hsa-miR-30e-5p, TRIGGER_3 = hsa-miR-1246, TRIGGER_ 4 = hsa-miR-30e-3p
trigger = ["UGAGAUGAAGCACUGUAGCUC", "UGUAAACAUCCUUGACUGGAAG", "AAUGGAUUUUUGGAGCAGG","CUUUCAGUCGGAUGUUUACAGC"]

#RBS
RBS = "AGAGGAGA"
#linker
linker = "AACCUGGCGGCAGCGCAAAAG"
#GFP
GFP = "atggtgagcaagggcgaggagctgttcaccggggtggtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtgtccggcgagggcgagggcgatgccacctacggcaagctgaccctgaagttcatctgcaccaccggcaagctgcccgtgccctggcccaccctcgtgaccaccctgacctacggcgtgcagtgcttcagccgctaccccgaccacatgaagcagcacgacttcttcaagtccgccatgcccgaaggctacgtccaggagcgcaccatcttcttcaaggacgacggcaactacaagacccgcgccgaggtgaagttcgagggcgacaccctggtgaaccgcatcgagctgaagggcatcgacttcaaggaggacggcaacatcctggggcacaagctggagtacaactacaacagccacaacgtctatatcatggccgacaagcagaagaacggcatcaaggtgaacttcaagatccgccacaacatcgaggacggcagcgtgcagctcgccgaccactaccagcagaacacccccatcggcgacggccccgtgctgctgcccgacaaccactacctgagcacccagtccgccctgagcaaagaccccaacgagaagcgcgatcacatggtcctgctggagttcgtgaccgccgccgggatcactctcggcatggacgagctgtacaagtaa".upper()
GFP = GFP.replace("T","U")


#write "ViennaRNA" for analysis with ViennaRNA
#"Nupack" for analysis with nupack
selection ="Nupack"

RBS.upper()
linker.upper()
GFP.upper()

paired = 9

unpaired = 9


In [3]:
from nupack import*

# toehold construction with Nupack
def nupack_design(a_seq,b_seq,paired,unpaired,rbs_seq,linker_seq):
    
    f_1 = open("results.txt","a+")
    f_2 = open("toeholds.txt","a+")
    ggg = Domain('GGG', name='GGG')

    # Define the two parts of the recognition sequence
    a = Domain(a_seq,name='a')
    b = Domain(b_seq,name='b')

    # Other Parts
    gua = Domain('GUA', name='gua')
    base_pairs = Domain('N6', name='base pairs')
    rbs = Domain(rbs_seq, name='RBS')
    aug = Domain('AUG', name='aug')
    linker = Domain(linker_seq, name='Linker')

    Toehold = TargetStrand([ggg, a, b, gua, base_pairs, rbs, ~base_pairs, aug, ~b, linker],name = 'Toehold')

    structure = '.3'
    structure += '.'+str(a.nt())
    structure += '('+str(b.nt())
    structure += '.'+str(gua.nt())
    structure += '('+str(base_pairs.nt())
    structure += '.'+str(rbs.nt())
    structure += ')'+str(base_pairs.nt())
    structure += '.'+str(aug.nt())
    structure += ')'+str(b.nt())
    structure += '.'+str(linker.nt())

    C1 = TargetComplex([Toehold], structure, name='C1')

    model = Model()

    complexes = [C1]

    design = complex_design(
        complexes=complexes,
        hard_constraints=[], soft_constraints=[],
        defect_weights=None, options=None, model=model
    )
    Toehold = str(Toehold).replace("NNNNNN","AUAUAU")
    result = design.run(trials=2) # run 2 independent design trials in the foreground
    f_1.write(str(Toehold)+'\t'+str(result)+'\n')
    f_2.write(str(Toehold)+'\n')
    f_1.close()
    f_2.close()
    
    return Toehold

In [17]:
import numpy as np
import matplotlib.pyplot as plt

# Analysis of a toehold with Nupack
def nupack_analysis(trigger, RBS, GFP, toehold):
    # initialization
    my_model = Model(material='RNA')
    trigger=Strand(trigger,name='trigger')
    complex_toehold_trigger = open("complex_toehold_trigger.txt","a+")
    complex_rbs_linker = open("complex_toehold_trigger.txt","a+")
    ### concentrations = open("concentrations.txt","a+")
    part_func = open("pfunc.txt","a+")
    sampled = open("sampled_structures.txt","a+")
    pairsf = open("pairs.txt","a+")
    mfe_struct = open("mfe.txt","a+") 
    
    #analysis toehold-trigger && rbs-linker
    b=toehold[toehold.find(RBS):toehold.find(GFP)]
    toehold = Strand(toehold,name='toehold_'+str(i+1))
    b =Strand(b,name='rbs-linker_'+str(i+1))
    
    set1 = ComplexSet(strands=[toehold, trigger], complexes=SetSpec(max_size=2))
    set2 = ComplexSet(strands=[b,b], complexes=SetSpec(max_size=1))
    
    complex_results1 = complex_analysis(complexes=set1, model=my_model, compute=['pfunc'])
    complex_results2 = complex_analysis(complexes=set2, model=my_model, compute=['pfunc'])
    complex_toehold_trigger.write(str(complex_results1)+"\n")
    complex_rbs_linker.write(str(complex_results2)+"\n")
    
    Concetrations = {}
    
    for initial_conc in np.linspace(1e-8, 1e-3):
        
        concentration_results1 = complex_concentrations(tube=set1, data=complex_results1,concentrations={toehold: initial_conc, trigger: initial_conc})
    
        # Extract Concetrations
        Concetrations_Dict = list(concentration_results1.tubes.values())[0].complex_concentrations
        
        # For every complex
        for c in Concetrations_Dict:
            if not c.name in Concetrations:
                Concetrations[c.name] = []
            Concetrations[c.name].append( Concetrations_Dict[c] )
    
    for c in Concetrations:
        plt.plot( np.linspace(1e-8, 1e-3), Concetrations[c] )
    plt.legend( Concetrations.keys() )
    plt.xlabel('Tube (M)')
    plt.ylabel('Equilibrium Concetration of Complexes (M)')
    plt.savefig('plots/toehold_'+str(toe_num))
    plt.close()
    
    ### concentrations.write(str(concentration_results1)+"\n")
    
    partition_function = pfunc(strands=toehold, model=my_model)
    part_func.write(str(partition_function)+"\n")
    
    sampled_structures = sample(strands=toehold, num_sample=3, model=my_model)
    sampled.write(str(sampled_structures)+"\n")

    probability_matrix = pairs(strands=[toehold,trigger ], model=my_model)
    pairsf.write(str(probability_matrix)+"\n")

    
    mfe_structures = mfe(strands=[toehold], model=my_model)
    mfe_struct.write(str(probability_matrix)+"\n")
    mfe_struct.write('Free energy of MFE proxy structure: %.2f kcal/mol' % mfe_structures[0].energy)
    mfe_struct.write('MFE proxy structure in dot-parens-plus notation: %s' % mfe_structures[0].structure)
    mfe_struct.write('MFE proxy structure as structure matrix:\n%s' % mfe_structures[0].structure.matrix())


In [18]:
from INPUT import*
from VIENNA_RNA import*

print("START NUPACK")

# design of the toeholds ~ many recognition sequences for each trigger and a certain value of unpaired bases 
# based on the rule: paired + unpaired
toe_num = 1
for j in range(len(trigger)):
    unpaired = 9
    while(paired+unpaired<=len(trigger[j])):
        for i in range(len(trigger[j])-paired-unpaired+1):
            trigger_j = Seq(trigger[j])
            toehold = nupack_design(str(trigger_j.reverse_complement()[i:i+unpaired+1]),str(trigger_j.reverse_complement()[i+unpaired+1:i+paired+unpaired]),paired,unpaired,RBS,linker)
            nupack_analysis(trigger[j],RBS,linker,toehold)
            toe_num += 1
        unpaired+=1

print("END")


START NUPACK
END


In [12]:
help(plt.close())

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.

